# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import db_connection_config as config

In [2]:
#conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn = psycopg2.connect(config.Config().connection_string(dbname="sparkifydb"))
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
print(song_files)

['c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAMQ128F1460CD3.json', 'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_

In [6]:
filepath = song_files[0]
filepath


'c:\\Users\\josep\\OneDrive\\Documentos\\projects\\UdacityDeng\\UDENG_L02P01\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json'

In [7]:
df = pd.read_json(filepath, lines=True) 
df.head()

num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  ARD7TVE1187B99BFB1              NaN               NaN   

   artist_location artist_name             song_id             title  \
0  California - LA      Casual  SOMZWCG12A8C13C480  I Didn't Mean To   

    duration  year  
0  218.93179     0

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:

song_data = df[["song_id", "title", "artist_id", "year", "duration"]].to_dict('records')[0]
song_data

{'song_id': 'SOMZWCG12A8C13C480',
 'title': "I Didn't Mean To",
 'artist_id': 'ARD7TVE1187B99BFB1',
 'year': 0,
 'duration': 218.93179}

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [15]:
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = 
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [27]:
log_files = get_files('data/log_data')

In [28]:
filepath = log_files[0]

In [34]:
df = pd.read_json(filepath, lines=True) 
df.head()


artist       auth firstName gender  itemInSession lastName     length  \
0     None  Logged In    Walter      M              0     Frye        NaN   
1     None  Logged In    Kaylee      F              0  Summers        NaN   
2  Des'ree  Logged In    Kaylee      F              1  Summers  246.30812   
3     None  Logged In    Kaylee      F              2  Summers        NaN   
4  Mr Oizo  Logged In    Kaylee      F              3  Summers  144.03873   

  level                           location method      page   registration  \
0  free  San Francisco-Oakland-Hayward, CA    GET      Home  1540919166796   
1  free        Phoenix-Mesa-Scottsdale, AZ    GET      Home  1540344794796   
2  free        Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong  1540344794796   
3  free        Phoenix-Mesa-Scottsdale, AZ    GET   Upgrade  1540344794796   
4  free        Phoenix-Mesa-Scottsdale, AZ    PUT  NextSong  1540344794796   

   sessionId          song  status             ts  \
0         38          None     200  1541105830796   
1        139          None     200  1541106106796   
2        139  You Gotta Be     200  1541106106796   
3        139          None     200  1541106132796   
4        139       Flat 55     200  1541106352796   

                                           userAgent  userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...      39  
1  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
2  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
3  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8  
4  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...       8

In [33]:
df.describe()

itemInSession      length  registration   sessionId  status  \
count      15.000000   11.000000  1.500000e+01   15.000000    15.0   
mean        2.600000  244.047339  1.540558e+12  127.000000   200.0   
std         2.772312   90.252182  3.253735e+08   45.639895     0.0   
min         0.000000  144.038730  1.540266e+12    9.000000   200.0   
25%         0.000000  188.956280  1.540345e+12  139.000000   200.0   
50%         2.000000  233.717100  1.540345e+12  139.000000   200.0   
75%         4.500000  245.837915  1.540968e+12  139.000000   200.0   
max         8.000000  440.267300  1.541034e+12  169.000000   200.0   

                 ts      userId  
count  1.500000e+01   15.000000  
mean   1.541108e+12   20.000000  
std    1.542550e+06   24.532777  
min    1.541106e+12    8.000000  
25%    1.541106e+12    8.000000  
50%    1.541107e+12    8.000000  
75%    1.541109e+12   26.000000  
max    1.541111e+12  101.000000

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.